In [98]:
import tensorflow as tf
import numpy as np
import random
#from google.colab import drive
from sklearn.model_selection import train_test_split

In [99]:
with open('iris.csv', 'r') as f:
  temp = np.genfromtxt(f, dtype='f4', delimiter=',')
for i in range(temp.shape[0]):
    if temp[i][-1] == 1.0:
        temp[i][-1] = 0.0
    else:
        temp[i][-1] = 1.0

In [100]:
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    # The examples are read at random, in no particular order
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        j = tf.constant(indices[i:min(i + batch_size, num_examples)])
        yield tf.gather(features, j), tf.gather(labels, j)

In [101]:
# Optimization algorithm Stochastic Gradient Descent
def sgd(param, grad, lr, batch_size):
  param.assign_sub(lr * grad)

In [102]:
# Loss Function
def cross_entropy(z, t):
  return tf.keras.losses.CategoricalCrossentropy()(t,z)
  #return  -(1/len(z))*tf.reduce_sum(tf.math.log(z)*t)


In [103]:
def model(x,w):
  # layer with softmax function
  u = np.hstack((np.ones((x.shape[0],1)), x))@w
  u_exp = tf.math.exp(u)
  z = u_exp/tf.reduce_sum(u_exp,axis=1,keepdims=True)
  # if softmax activation is to be used,
  # z = tf.nn.softmax(u)
  return z

In [104]:
# Data Prep

X = temp[:,0:-1]
# One-hot output layer encodding
from sklearn.preprocessing import LabelBinarizer
LB = LabelBinarizer()
labels = LB.fit_transform(temp[:,-1])
print(labels.shape)

# K: Nsamples, d: featureDimension, N: Nclasses
K,d = X.shape
N = labels.shape[1]

# partition into 80/20% training/testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, labels, train_size=0.8, shuffle=True, random_state=3, stratify=labels)

(150, 1)


In [105]:
from sklearn.preprocessing import LabelBinarizer
LB = LabelBinarizer()
labels = LB.fit_transform(temp[:,-1])
print(labels.shape)

(150, 1)


In [106]:
# Hyperparameters

batch_size = 10
lr = 0.03 # learning rate
num_epochs = 100

In [107]:
# Learning

w = tf.Variable(tf.random.normal(shape=(d+1,N)), trainable=True)
for epoch in range(num_epochs):
  for x, y in data_iter(batch_size, X_train, y_train):
    # Feed-forward model
    with tf.GradientTape() as g:
      l = cross_entropy(model(x, w),y)
    # Compute gradient on l with respect to w
    dw = g.gradient(l, w)
    # Update parameters using their gradient
    sgd(w, dw, lr, batch_size)
  # After one epoch, Evaluate resubstitution loss
  train_loss = cross_entropy(model(tf.constant(X_train), w),y_train)
  print(f'epoch {epoch + 1}, loss {float(tf.reduce_mean(train_loss)):f}')

TypeError: Missing required positional argument

In [ ]:
# Performance Evaluation
# Z_test is the linear model output (real #)
Z_test = model(X_test,w)
# idx is an array consisting of the index of largest output of each sample
idx = np.argmax(Z_test,axis=1)
# y_pred is one hot encoding of the predicted output
y_pred = np.zeros((idx.size, N))
y_pred[np.arange(idx.size),idx] = 1
# Cmat is the confusion matrix
Cmat = y_test.T@y_pred
Accuracy = np.round(100*np.sum(np.diagonal(Cmat))/np.sum(Cmat),2)
print('Confusion Matrix= \n', Cmat)
print('Accuracy = ', Accuracy, '%' )